In [16]:
import asyncio
import os
from random import randint
from typing import Annotated
from dotenv import load_dotenv
from pydantic import Field

from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI

from langchain.agents import create_agent

In [17]:
load_dotenv()

True

In [18]:
@tool
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")]
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."



In [19]:
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0,
)

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    prompt="You are a helpful weather agent.",
)

C:\Users\armoshar\AppData\Local\Temp\ipykernel_77556\4096649293.py:9: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [20]:
query = "What's the weather like in Seattle?"

result = agent.invoke(
    {"messages": [("user", query)]}
)

result["messages"][-1].content


"Right now in Seattle, it's sunny with a high of 28°C. Sounds like a beautiful day!"

In [21]:
import asyncio

async def stream_agent(query: str):
    print("Agent:", end=" ", flush=True)
    async for event in agent.astream_events(
        {"messages": [("user", query)]},
        version="v2",
    ):
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if getattr(chunk, "content", None):
                print(chunk.content, end="", flush=True)
    print()

await stream_agent("What's the weather like in Portland?")


Agent: Sure! Let me check the current weather in Portland for you.Right now in Portland, it's cloudy with a high temperature of 29°C.
